# Linear Regression

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.stats.stattools import durbin_watson
from scipy.stats import shapiro
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from statsmodels.stats.outliers_influence import variance_inflation_factor

## Data preprocessing

### Data collection

In [2]:
df = pd.read_csv('data/freedom_index.csv')

In [ ]:
df

In [ ]:
df.describe()

### Data cleansing

#### Drop unuse columns

In [5]:
unused_cols = ['country']

In [6]:
df = df.drop(columns=unused_cols)

#### Define target

In [7]:
target = 'score'

In [ ]:
df[target].hist()

#### Check data type

In [ ]:
df.dtypes

#### Clean missing value

In [ ]:
df.isnull().sum()

In [11]:
group_cols = ['region']

# Drop rows with no target value.
df = df.dropna(subset=[target])

for col in [x for x in df.columns if x not in group_cols + ['target']]:
    if (df[col].dtype == 'int64') | (df[col].dtype == 'float64'):
        # Fill mean value of its category for number columns.
        df.loc[:, col] = df.groupby(group_cols)[col].transform(
            lambda x: x.fillna(x.mean())
        )
    elif df[col].dtype == 'object':
        # Fill most common value of its category for catgorical columns.
        df.loc[:, col] = df.groupby(group_cols)[col].transform(
            lambda x: x.fillna(x.mode()[0]) if not x.mode().empty else x
        )

In [ ]:
df

#### Transform categorical columns

In [13]:
cate_cols = ['region']
df = pd.get_dummies(df, columns=cate_cols, prefix=cate_cols, drop_first=True)

In [ ]:
df

In [15]:
df = df.apply(lambda x: x.astype(int) if x.dtype == 'bool' else x)

In [ ]:
df

#### Assign target and features

In [17]:
features = [x for x in df.columns if x != target]

In [ ]:
features

In [19]:
y = df[target]
X = df[features]

In [ ]:
y

In [ ]:
X

### Split dataset

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
X_train

In [ ]:
y_train

### Feature selection

#### Step-wise

In [25]:
def stepwise(X, y, threshold_in=0.05, threshold_out=0.05):
    """
    Stepwise regression for Linear Regression model.
    """
    included = []
    
    while True:
        changed = False
        
        # Forward Step
        excluded = list(set(X.columns) - set(included))
        new_pval = pd.Series(index=excluded)
        
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included + [new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        
        best_pval = new_pval.min()
        
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed = True
            
            print(f'Add {best_feature} with p-value {best_pval}')
        
        # Backward Step
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() 
        
        if worst_pval > threshold_out:
            changed = True
            worst_feature = pvalues.idxmax()
            included.remove(worst_feature)
            
            print(f'Drop {worst_feature} with p-value {worst_pval}')
        
        if not changed:
            break

    return included

In [ ]:
features = stepwise(X_train, y_train)

In [ ]:
features

In [28]:
X_train = X_train[features]
X_test = X_test[features]

In [ ]:
X_train

### Feature Engineering

#### Standardization

In [30]:
scaler = StandardScaler()

In [31]:
train_index = X_train.index
X_train_scale = pd.DataFrame(scaler.fit_transform(X_train), columns=features)
X_train_scale.index = train_index

In [ ]:
X_train

In [33]:
test_index = X_test.index
X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=features)
X_test_scale.index = test_index

## Model

### First training

In [34]:
model = sm.OLS(y_train, sm.add_constant(X_train_scale)).fit()

In [ ]:
model.summary()

In [36]:
y_pred = model.predict(sm.add_constant(X_test_scale))
residuals = y_test - y_pred

### Check assumption

#### Linearity

In [ ]:
plt.scatter(y_pred, residuals)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.title('Residuals vs Predicted Values (Linearity Check)')
plt.show()

In [ ]:
for feature in features:
    plt.scatter(X_test_scale[feature], residuals)
    plt.axhline(y=0, color='r', linestyle='--')
    plt.title(f'Residuals vs {feature} (Linearity Check)')
    plt.xlabel(feature)
    plt.ylabel('Residuals')
    plt.show()

#### Independence

In [ ]:
dw_test = durbin_watson(residuals)
print(f"Durbin-Watson test statistic: {dw_test}")

#### Homoscedasticity

In [ ]:
plt.scatter(y_pred, residuals)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.title('Residuals vs. Predicted Values (Homoscedasticity Check)')
plt.show()

#### Normality of residuals

In [ ]:
sm.qqplot(residuals, line='s')
plt.title('Normality of Residuals')
plt.show()

In [ ]:
plt.hist(residuals, bins=30, edgecolor='k')
plt.title('Residual Histogram')
plt.xlabel('Residuals')
plt.ylabel('Frequency')
plt.show()

In [ ]:
shapiro_test = shapiro(residuals)
print(f"Shapiro-Wilk test for normality: {shapiro_test}")

#### Multicollinearity

In [ ]:
X_vif = sm.add_coanstant(X_train_scale[features])
vif = pd.DataFrame()
vif['Features'] = X_train_scale[features].columns
vif['VIF'] = [variance_inflation_factor(X_vif, i + 1) for i in range(len(X_train_scale[features].columns))]
print(vif)

### Fix assumption

#### Normality of Residuals

In [45]:
y_train = np.log(y_train + 1)
y_test = np.log(y_test + 1)

In [ ]:
y_train

### Retrain model

#### Feature selection

In [ ]:
features = stepwise(X_train, y_train)

In [48]:
X_train = X_train[features]
X_test = X_test[features]

#### Feature engineering

In [49]:
scaler = StandardScaler()

In [50]:
train_index = X_train.index
X_train_scale = pd.DataFrame(scaler.fit_transform(X_train), columns=features)
X_train_scale.index = train_index

In [ ]:
X_train

In [52]:
test_index = X_test.index
X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=features)
X_test_scale.index = test_index

#### Model training

In [53]:
model = sm.OLS(y_train, sm.add_constant(X_train_scale)).fit()

In [ ]:
model.summary()

In [55]:
y_pred = model.predict(sm.add_constant(X_test_scale))
residuals = y_test - y_pred

### Recheck assumption

#### Normality of residuals

In [ ]:
sm.qqplot(residuals, line='s')
plt.title('Normality of Residuals')
plt.show()

In [ ]:
plt.hist(residuals, bins=30, edgecolor='k')
plt.title('Residual Histogram')
plt.xlabel('Residuals')
plt.ylabel('Frequency')
plt.show()

In [ ]:
shapiro_test = shapiro(residuals)
print(f"Shapiro-Wilk test for normality: {shapiro_test}")

## Evaluate

In [ ]:
y_train.hist()
plt.title('Train Target Distribution')
plt.show()

In [ ]:
y_pred_train = model.predict(sm.add_constant(X_train))
mean_squared_error(y_pred_train, y_train)

In [ ]:
y_test.hist()
plt.title('Test Target Distribution')
plt.show()

In [ ]:
mean_squared_error(y_pred, y_test)

In [ ]:
plt.scatter(y_pred, y_test)
plt.xlabel('Predicted Values')
plt.ylabel('Actual Values')
plt.title('Predicted Values vs. Actual Values')
plt.show()

## Test run

In [ ]:
features

In [65]:
input = {
    'business_freedom': 68.5,
    'tax_burden': 68.0,
    'monetary_freedom': 71.1,
    'trade_freedom': 67.4,
    'judicial_effectiveness': 32.9,
    'government_spending': 33.9,
    'fiscal_health': 29.9,
    'investment_freedom': 65.0,
    'labor_freedom': 48.6
}

In [66]:
input_df = pd.DataFrame(input, index=[0])

In [ ]:
input_df

In [68]:
input_df = pd.DataFrame(scaler.transform(input_df), columns=features)

In [ ]:
input_df

In [70]:
log_result = model.predict(sm.add_constant(input_df, has_constant='add'))

In [ ]:
log_result

In [72]:
result = np.exp(log_result)

In [ ]:
result